In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd

In [2]:
df = pd.read_csv('fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
df.shape

(35887, 3)

In [3]:
X_train = []
Y_train = []
X_test = []
Y_test = []
X_val = []
Y_val = []
for i, row in df.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage']=='Training':
        X_train.append(np.array(k))
        Y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        Y_test.append(row['emotion'])
    elif row['Usage'] == 'PrivateTest':
        X_val.append(np.array(k))
        Y_val.append(row['emotion'])

In [5]:
X_train = np.array(X_train, dtype = 'uint8')
Y_train = np.array(Y_train, dtype = 'uint8')
X_test = np.array(X_test, dtype = 'uint8')
Y_test = np.array(Y_test, dtype = 'uint8')

In [6]:
import keras
from keras.utils import to_categorical
Y_train= to_categorical(Y_train, num_classes=7)
Y_test = to_categorical(Y_test, num_classes=7)

In [7]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [8]:
from keras.preprocessing.image import ImageDataGenerator 
datagen = ImageDataGenerator( rescale=1./255,rotation_range = 10,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode = 'nearest')

In [9]:
testgen = ImageDataGenerator(rescale=1./255)
datagen.fit(X_train)

In [10]:
batch_size = 64
train_flow = datagen.flow(X_train, Y_train, batch_size=batch_size) 
test_flow = testgen.flow(X_test, Y_test, batch_size=batch_size)

In [3]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [53]:
#CNN model
model = Sequential()
model.add(Conv2D(64,(3,3), activation = 'relu', padding = 'same', input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(256,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(512,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
model.add(Dense(7,activation = 'softmax'))


opt = Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [54]:
num_epochs = 100  
callbacks = [EarlyStopping(monitor='val_loss', patience=10), ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
history = model.fit(train_flow,steps_per_epoch=len(X_train) / batch_size, epochs=num_epochs,  verbose=1,  validation_data=test_flow)

Epoch 1/100
448/448 [==============================] - 53s 115ms/step - loss: 1.9456 - accuracy: 0.2523 - val_loss: 2.0690 - val_accuracy: 0.1700
Epoch 2/100
448/448 [==============================] - 52s 116ms/step - loss: 1.6958 - accuracy: 0.3197 - val_loss: 1.6891 - val_accuracy: 0.3600
Epoch 3/100
448/448 [==============================] - 53s 117ms/step - loss: 1.5968 - accuracy: 0.3677 - val_loss: 1.5839 - val_accuracy: 0.3940
Epoch 4/100
448/448 [==============================] - 52s 116ms/step - loss: 1.5151 - accuracy: 0.4079 - val_loss: 1.5504 - val_accuracy: 0.4352
Epoch 5/100
448/448 [==============================] - 52s 117ms/step - loss: 1.4342 - accuracy: 0.4421 - val_loss: 1.4072 - val_accuracy: 0.4709
Epoch 6/100
448/448 [==============================] - 52s 117ms/step - loss: 1.3567 - accuracy: 0.4761 - val_loss: 1.3040 - val_accuracy: 0.5029
Epoch 7/100
448/448 [==============================] - 52s 117ms/step - loss: 1.2906 - accuracy: 0.5083 - val_loss: 1.2800 -

Epoch 57/100
448/448 [==============================] - 53s 117ms/step - loss: 0.3574 - accuracy: 0.8688 - val_loss: 1.3125 - val_accuracy: 0.6551
Epoch 58/100
448/448 [==============================] - 52s 117ms/step - loss: 0.3584 - accuracy: 0.8685 - val_loss: 1.3426 - val_accuracy: 0.6729
Epoch 59/100
448/448 [==============================] - 52s 117ms/step - loss: 0.3402 - accuracy: 0.8744 - val_loss: 1.3235 - val_accuracy: 0.6704
Epoch 60/100
448/448 [==============================] - 52s 117ms/step - loss: 0.3292 - accuracy: 0.8790 - val_loss: 1.3213 - val_accuracy: 0.6751
Epoch 61/100
448/448 [==============================] - 52s 117ms/step - loss: 0.3217 - accuracy: 0.8802 - val_loss: 1.3513 - val_accuracy: 0.6734
Epoch 62/100
448/448 [==============================] - 52s 117ms/step - loss: 0.3172 - accuracy: 0.8836 - val_loss: 1.3698 - val_accuracy: 0.6648
Epoch 63/100
448/448 [==============================] - 52s 117ms/step - loss: 0.2997 - accuracy: 0.8916 - val_loss: 1

In [ ]:
model.save('emotiondet.h5')

In [4]:
from tensorflow.keras.models import load_model
model = load_model('emotiondet.h5')

In [5]:
import cv2
from tensorflow.keras.utils import img_to_array
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [8]:
cap = cv2.VideoCapture(0)
while True:
    res, frame = cap.read()
    height, width , channel = frame.shape
    gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray_image)
    try:
        for (x,y,w,h) in faces:
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)
            roi_gray = gray_image[y-5:y+h+5,x-5:x+w+5]
            roi_gray=cv2.resize(roi_gray,(48,48))
            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis = 0)
            image_pixels /= 255
            predictions = model.predict(image_pixels)
            max_index = np.argmax(predictions[0])
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]
            frame = cv2.putText(frame,emotion_prediction,(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
    except:
        pass
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key==27:
        break
cv2.destroyAllWindows()
cap.release()


1/1 [==============================] - 0s 18ms/step
